# Script to get the jacobian matrix which under the .ipynb format

In [1]:
import sympy as sym
import numpy as np

In [2]:
x, y, z, w, h, l, v, yaw, ry_rate = sym.symbols("x y z w h l v yaw ry_rate")
next_x, next_y, next_z, next_w, next_h, next_l, next_v, next_yaw, next_ry_rate = sym.symbols("next_x next_y next_z next_w next_h next_l next_v next_yaw next_ry_rate")
dt = sym.symbols("dt")

In [3]:
next_z, next_w, next_h, next_l, next_v, next_ry_rate = z, w, h, l, v, ry_rate
next_yaw = yaw + ry_rate * dt

In [4]:
vt, yawt, ita, t = sym.symbols("vt yawt ita t")
"""
yawt = yaw + ry_rate * (ita - (t - 1) * dt)
next_x = x + sym.integrate(v * sym.cos(yawt), (ita, (t - 1) * dt, t * dt))
next_y = y + sym.integrate(v * sym.sin(yawt), (ita, (t - 1) * dt, t * dt))
"""
# warp raw transition time from [(t-1)*dt, t*dt] to [0, dt], for quick solution
vt = v
yawt = yaw + ry_rate * (ita)
next_x = x + sym.integrate(vt * sym.cos(yawt), (ita, 0, dt))
next_y = y + sym.integrate(vt * sym.sin(yawt), (ita, 0, dt))

# Analytical solutions for displacements can also be obtained

In [5]:
next_x

x + Piecewise((-v*sin(yaw)/ry_rate + v*sin(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*v*cos(yaw), True))

In [6]:
next_y

y + Piecewise((v*cos(yaw)/ry_rate - v*cos(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*v*sin(yaw), True))

In [7]:
funcs = sym.Matrix([next_x, next_y, next_z, next_w, next_l, next_h, next_v, next_yaw, next_ry_rate])
args = sym.Matrix([x, y, z, w, l, h, v, yaw, ry_rate])
res = funcs.jacobian(args)

In [8]:
res

Matrix([
[1, 0, 0, 0, 0, 0, Piecewise((-sin(yaw)/ry_rate + sin(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*cos(yaw), True)), Piecewise((-v*cos(yaw)/ry_rate + v*cos(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (-dt*v*sin(yaw), True)), Piecewise((dt*v*cos(dt*ry_rate + yaw)/ry_rate + v*sin(yaw)/ry_rate**2 - v*sin(dt*ry_rate + yaw)/ry_rate**2, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (0, True))],
[0, 1, 0, 0, 0, 0,  Piecewise((cos(yaw)/ry_rate - cos(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*sin(yaw), True)),  Piecewise((-v*sin(yaw)/ry_rate + v*sin(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*v*cos(yaw), True)), Piecewise((dt*v*sin(dt*ry_rate + yaw)/ry_rate - v*cos(yaw)/ry_rate**2 + v*cos(dt*ry_rate + yaw)/ry_rate**2, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (0, True))],
[0, 0, 1, 0, 0, 0,                               

In [9]:
mea_funcs = sym.Matrix([x, y, z, w, l, h, v * sym.cos(yaw), v * sym.sin(yaw), yaw])
mea_res = mea_funcs.jacobian(args)

In [10]:
mea_res

Matrix([
[1, 0, 0, 0, 0, 0,        0,           0, 0],
[0, 1, 0, 0, 0, 0,        0,           0, 0],
[0, 0, 1, 0, 0, 0,        0,           0, 0],
[0, 0, 0, 1, 0, 0,        0,           0, 0],
[0, 0, 0, 0, 1, 0,        0,           0, 0],
[0, 0, 0, 0, 0, 1,        0,           0, 0],
[0, 0, 0, 0, 0, 0, cos(yaw), -v*sin(yaw), 0],
[0, 0, 0, 0, 0, 0, sin(yaw),  v*cos(yaw), 0],
[0, 0, 0, 0, 0, 0,        0,           1, 0]])